
# Preparing data

In [ ]:
import os
import pandas as pd

from common_code import data_folder

## Product and cost data

In [ ]:
products_df = pd.read_csv(f'{data_folder}/products.csv')

products_df.head()

In [ ]:
products_df.info()

In [ ]:
products_df = pd.read_csv(f'{data_folder}/products.csv')
products_df = products_df.drop('Brand', axis='columns')

products_df.head()

In [ ]:
costs_df = pd.read_csv(f'{data_folder}/standard costs.csv')

costs_df

In [ ]:
costs_df = pd.read_csv(f'{data_folder}/standard costs.csv', sep='\t')

costs_df

In [ ]:
costs_df = pd.read_csv(f'{data_folder}/standard costs.csv', sep='\t')
costs_df = costs_df.rename(columns={'Standard Unit Cost': 'Unit Cost'})
costs_df = costs_df[['ProductID', 'Unit Cost']]

costs_df

In [ ]:
costs_df.info()

## Sales data

In [ ]:
def get_sales(file_name): 
    df = pd.read_excel(f'{data_folder}/{file_name}')
    return df

In [ ]:
get_sales('Q1Sales.xlsx')

In [ ]:
def get_sales(file_name): 
    df = pd.concat(
        pd.read_excel(f'{data_folder}/{file_name}', sheet_name=None), 
        ignore_index=True
    )

    return df

In [ ]:
sales_df = get_sales('Q1Sales.xlsx')

sales_df

In [ ]:
sales_df.nunique()

In [ ]:
sales_df.isna().sum()

In [ ]:
sales_df.duplicated().sum()

In [ ]:
def get_sales(file_name):
    df = pd.concat(
        pd.read_excel(f'{data_folder}/{file_name}', sheet_name=None), 
        ignore_index=True
    )
    
    df = df.drop_duplicates()
    df = df.convert_dtypes()
    
    df = df[[
        'InvoiceNo', 'Channel', 'ProductID', 
        'Date', 'Unit Price', 'Quantity', 'Total' 
    ]]
    
    df = df.merge(products_df, on='ProductID', validate='many_to_one')
    df = df.merge(costs_df, on='ProductID', validate='many_to_one')
    df = df.sort_values('InvoiceNo')
    
    return df

In [ ]:
sales_df = get_sales('Q1Sales.xlsx')

sales_df

In [ ]:
def get_sales(file_name):
    # Reads sales Excel file, cleans and merges it with products and costs
    # data, and returns a new DataFrame.
    
    df = pd.concat(
        pd.read_excel(f'{data_folder}/{file_name}', sheet_name=None), 
        ignore_index=True
    )
    
    df = df.drop_duplicates()
    df = df.convert_dtypes()
    
    # keep useful columns
    df = df[[
        'InvoiceNo', 'Channel', 'ProductID', 
        'Date', 'Unit Price', 'Quantity', 'Total' 
    ]]
    
    # merge with products and costs data
    df = df.merge(products_df, on='ProductID')
    df = df.merge(costs_df, on='ProductID')
    df = df.sort_values('InvoiceNo')
    
    # check for missing values in any of the columns
    # and for sale events spilling across quarters
    assert df.isna().sum().sum() == 0, 'Data contains NA values'
    assert len(df['Date'].dt.quarter.unique()) == 1, 'Data from multiple quarters'
    
    return df

In [ ]:
os.listdir(data_folder)

In [ ]:
sales_df = pd.concat(
    [get_sales(name) for name in os.listdir(data_folder) if name.endswith('.xlsx')], 
    ignore_index=True
)

In [ ]:
sales_df

In [ ]:
sales_df['Gross Profit'] = (sales_df['Total'] - 
                           (sales_df['Quantity'] * sales_df['Unit Cost']))

sales_df['Profit per Unit'] = sales_df['Gross Profit'] / sales_df['Quantity']
sales_df['Margin per Unit'] = ((sales_df['Profit per Unit'] / 
                                sales_df['Unit Price']) * 100)

In [ ]:
sales_df.to_csv(f'{data_folder}/sales2020.csv', index=False)